### Retrieving the Financial Data

In [84]:
import fitz
import re
import pandas as pd

def read_pdf_file(file_path, start_page=1, end_page=100):
    pdf_document = fitz.open(file_path)
    total_pages = pdf_document.page_count
    pdf_text = ''

    for page_num in range(start_page - 1, end_page):
        pdf_page = pdf_document.load_page(page_num)
        pdf_text += pdf_page.get_text()

    return pdf_text



def format_data(extracted_data):
    data = {}
    lines = extracted_data.split('\n')
    
    for i, line in enumerate(lines):
        numerical_data = re.findall(r'(-?\d{1,3}(?:\s\d{3})*\s\d{3})', line)
        numerical_data = ' '.join(numerical_data).replace(' ', '')
        numerical_data = int(numerical_data) if numerical_data else None
        
        if numerical_data:
            variable_name = None
            
            if i > 0 and not re.search(r'\d', lines[i-1]):
                variable_name = lines[i-1]
            elif i < len(lines) - 1 and not re.search(r'\d', lines[i+1]):
                variable_name = lines[i+1]
            
            if variable_name is not None:
                data[variable_name] = numerical_data
    
    
    split_key_first = 'Årsresultat etter minoritetsinteresser'
    split_key_second = 'Årsresultat'
    
    # check if the first split key is in the data
    
    if split_key_first in data.keys():
        split_key = split_key_first
    else:
        split_key = split_key_second
    
    
    
    keys = list(data.keys())
    split_index = keys.index(split_key) + 1
    
    income_statement, balance_sheet = {k: data[k] for k in keys[:split_index]}, {k: data[k] for k in keys[split_index:]}
    
    return income_statement, balance_sheet


def get_data(file_path, start_page=1, end_page=100):
    extracted_data = read_pdf_file(file_path, start_page, end_page)
    income_statement, balance_sheet = format_data(extracted_data)
    
    return income_statement, balance_sheet

In [88]:
years = range(2014, 2022)
import json

all_years = {}

for year in years:
    
    input_path = f'elkem_data/pdf/brreg/{year}.pdf'
    output_path = f'elkem_data/json/{year}.json'
    
    income_statement, balance_sheet = get_data(input_path, start_page=6, end_page=11)
    
    with open(output_path, 'w') as f:
        json.dump({'income_statement': income_statement, 'balance_sheet': balance_sheet}, f, indent=4)
    
    all_years[year] = {'income_statement': income_statement, 'balance_sheet': balance_sheet}

with open('elkem_data/json/all_years.json', 'w') as f:
    json.dump(all_years, f, indent=4)

In [86]:
all_years

{2014: {'income_statement': {'Salgsinntekt': 8561833000,
   'Annen driftsinntekt': 90519000,
   'Sum inntekter': 8652352000,
   'Kostnader': 7788724000,
   'Varekostnad': 4251156000,
   'Lønnskostnad': 1358684000,
   'Avskrivning på varige driftsmidler og immaterielle eiendeler': 394347000,
   'Nedskrivning av varige driftsmidler og immaterielle eiendeler': 730000,
   'Other operating expences': 1982864000,
   'Other gains and losses': 53155000,
   'Sum kostnader': 8040936000,
   'Driftsresultat': 611416000,
   'Finansinntekter og finanskostnader': 417621000,
   'Inntekt på investering i datterselskap og tilknyttet selskap': -262045000,
   'Sum finansinntekter': -262045000,
   'Finance expences net': -42953000,
   'Sum finanskostnader': -42953000,
   'Netto finans': -219092000,
   'Ordinært resultat før skattekostnad': 392324000,
   'Skattekostnad på ordinært resultat': 169802000,
   'Ordinært resultat etter skattekostnad': 222522000,
   'Profit for the year from discontinued operation

### Formatting

In [102]:
import json



translation_mapping = {
"Salgsinntekt": "Revenue",
"Annen driftsinntekt": "Other operating income",
"Kostnader": "Expenses",
"Varekostnad": "Cost of goods sold",
"Lønnskostnad": "Employee benefit expenses",
"Avskrivning på varige driftsmidler og immaterielle eiendeler": "Amortisation and depreciation",
"Nedskrivning av varige driftsmidler og immaterielle eiendeler": "Impairment losses",
"Annen driftskostnad": "Other operating expenses",
"Driftsresultat": "Operating profit",
"Finansinntekter og finanskostnader": "Finance income and expenses",
"Annen finansinntekt": "Finance income",
"Sum finansinntekter": "Total finance income",
"Annen finanskostnad": "Finance expenses",
"Netto finans": "Net finance",
"Ordinært resultat før skattekostnad": "Profit before tax",
"Skattekostnad på ordinært resultat": "Income tax (expense) benefit",
"Ordinært resultat etter skattekostnad": "Profit after tax",
"Årsresultat": "Annual profit",
"Sum inntekter": "Total revenue",
"Sum kostnader": "Total expenses",
"Sum finansinntekter og finanskostnader": "Total finance income and expenses",
"Inntekt på investering i datterselskap og tilknyttet selskap" : "Income from investments in subsidiaries and associated companies",
"Sum finanskostnader": "Total finance expenses",
"Årsresultat etter minoritetsinteresser" : "Annual profit after minority interests",


"Utsatt skattefordel": "Deferred tax asset",
"Goodwill": "Goodwill",
"Other intangible assets": "Other intangible assets",
"Sum immaterielle eiendeler": "Total intangible assets",
"Varige driftsmidler": "Tangible fixed assets",
"Property, plant and equipment": "Property, plant and equipment",
"Right-of-use assets": "Right-of-use assets",
"Sum varige driftsmidler": "Total tangible fixed assets",
"Finansielle anleggsmidler": "Financial fixed assets",
"Investments in equity accounted companies": "Investments in equity accounted companies",
"Derivatives": "Derivatives",
"Other assets": "Other assets",
"Sum finansielle anleggsmidler": "Total financial fixed assets",
"Sum anleggsmidler": "Total fixed assets",
"Omløpsmidler": "Current assets",
"Varer": "Inventories",
"Sum varer": "Total inventories",
"Fordringer": "Receivables",
"Kundefordringer": "Trade receivables",
"Restricted deposits": "Restricted deposits",
"Sum fordringer": "Total receivables",
"Bankinnskudd, kontanter og lignende": "Cash and cash equivalents",
"Sum bankinnskudd, kontanter og lignende": "Total cash and cash equivalents",
"Sum omløpsmidler": "Total current assets",
"SUM EIENDELER": "TOTAL ASSETS",
"BALANSE - EGENKAPITAL OG GJELD": "BALANCE - EQUITY AND LIABILITIES",
"Selskapskapital": "Share capital",
"Other paid in capital": "Other paid in capital",
"Sum innskutt egenkapital": "Total contributed equity",
"Opptjent egenkapital": "Retained earnings",
"Annen egenkapital": "Other equity",
"Minoritetsinteresser": "Minority interests",
"Sum opptjent egenkapital": "Total retained earnings",
"Sum egenkapital": "Total equity",
"Gjeld": "Debt",
"Utsatt skatt": "Deferred tax",
"Sum avsetninger for forpliktelser": "Total provisions for liabilities",
"Annen langsiktig gjeld": "Other long-term debt",
"Gjeld til kredittinstitusjoner": "Debt to credit institutions",
"Employee benefit obligations": "Employee benefit obligations",
"Provisions and other liabilities": "Provisions and other liabilities",
"Sum annen langsiktig gjeld": "Total other long-term debt",
"Sum langsiktig gjeld": "Total long-term debt",
"Kortsiktig gjeld": "Short-term debt",
"Leverandørgjeld": "Accounts payable",
"Betalbar skatt": "Income tax payable",
"Bills payable": "Bills payable",
"Sum kortsiktig gjeld": "Total short-term debt",
"Sum gjeld": "Total debt",
"SUM EGENKAPITAL OG GJELD": "TOTAL EQUITY AND LIABILITIES",
"Paid-in capita": "Paid-in capital",
"Inventories": "Inventories",
"Trade receivables": "Trade receivables",
"Bankinnskudd, kontanter og lignende": "Cash and cash equivalents",
"Restricted deposits": "Restricted deposits",
"Cash and cash equivalents": "Cash and cash equivalents",
"Income tax payables": "Income tax payables",
"Interest-bearing liabilities": "Interest-bearing liabilities",
"Investements in qeuity accounted companies": "Investments in equity accounted companies",
"Pensjonsforpliktelser": "Pension obligations",
"Øvrig langsiktig gjeld": "Other long-term debt",
"Forskning og utvikling": "Research and development",
"rettigheter": "Concessions, patents, licenses, trademarks and similar rights",
"Sum investeringer": "Total investments",
"Tomter, bygninger og annen fast eiendom": "Land, buildings and other fixed assets",

}



def translate_data(data, translation_mapping):
    translated_data = {}

    for key, value in data.items():
        if key in translation_mapping:
            translated_key = translation_mapping[key]
            translated_data[translated_key] = value
        else:
            translated_data[key] = value

    return translated_data

def translate_financial_statements(year_data, translation_mapping):
    translated_year_data = {
        year: {
            financial_statement: translate_data(data, translation_mapping)
            for financial_statement, data in year_data.items()
        }
        for year, year_data in year_data.items()
    }

    return translated_year_data


with open('elkem_data/json/all_years.json', 'w') as f:
    json.dump(all_years, f, indent=4)

In [104]:
import json

input_json = all_years

# Extract income statement and balance sheet keys for the year 2021
years = set(input_json.keys())
format_year = max(years)

years.remove(format_year)

income_statement_keys = list(input_json[format_year]["income_statement"].keys())
balance_sheet_keys = list(input_json[format_year]["balance_sheet"].keys())

formatted_json = {}



for year in sorted(years):
    formatted_json[year] = {
        "income_statement": {},
        "balance_sheet": {},
    }

    for key in income_statement_keys:
        if key in input_json[year]["income_statement"]:
            formatted_json[year]["income_statement"][key] = input_json[year]["income_statement"][key]

    for key in balance_sheet_keys:
        if key in input_json[year]["balance_sheet"]:
            formatted_json[year]["balance_sheet"][key] = input_json[year]["balance_sheet"][key]

# Add the year 2021 data as it is
formatted_json[format_year] = input_json[format_year]

# order the year keys in the JSON

formatted_json

# Save the formatted JSON to a new file
with open("elkem_data/json/all_years_formatted.json", "w") as outfile:
    json.dump(formatted_json, outfile, indent=4)


In [106]:
import pandas as pd
import json

json_data = all_years

def json_to_dataframes(json_data):
    data = json_data

    income_statements = {}
    balance_sheets = {}

    for year, content in data.items():
        income_statements[year] = content["income_statement"]
        balance_sheets[year] = content["balance_sheet"]

    income_statement_df = pd.DataFrame(income_statements)
    balance_sheet_df = pd.DataFrame(balance_sheets)

    return income_statement_df, balance_sheet_df

income_statement_df, balance_sheet_df = json_to_dataframes(json_data)

income_statement_df
balance_sheet_df


,2014,2015,2016,2017,2018,2019,2020,2021
Research and development,9123000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Concessions, patents, licenses, trademarks and similar rights",160884000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Deferred tax asset,320882000.0,3.239690e+08,6.734800e+07,8.958400e+07,6.000000e+07,6.600000e+07,9.600000e+07,4.800000e+07
Goodwill,160734000.0,2.440880e+08,3.426450e+08,3.263230e+08,3.420000e+08,4.660000e+08,9.190000e+08,9.410000e+08
Total intangible assets,642500000.0,1.211550e+09,1.103006e+09,1.135257e+09,1.324000e+09,1.309000e+09,2.334000e+09,2.591000e+09
...,...,...,...,...,...,...,...,...
Provisions and other liabilities,NaN,NaN,NaN,NaN,NaN,NaN,1.063000e+09,1.655000e+09
Other intagible assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.602000e+09
Paid-in capital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.097000e+09
Interest-bearing liabilities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.972000e+09


### Saving to Excel

In [75]:
import json
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

all_years = json.load(open('elkem_data/json/all_years_formatted.json'))


workbook = Workbook()
workbook.remove(workbook.active)

# Create Income Statement and Balance Sheet worksheets
income_statement_ws = workbook.create_sheet('Income Statements')
balance_sheet_ws = workbook.create_sheet('Balance Sheets')

combined_income_statement_ws = workbook.create_sheet('Combined Income Statements')
combined_balance_sheet_ws = workbook.create_sheet('Combined Balance Sheets')


# write the income_statement_df and balance_sheet_df to excel

com

current_col = 1

for year in all_years:
    income_statement_df = pd.DataFrame(all_years[year]['income_statement'].items(), columns=['Variable', 'Value'])
    balance_sheet_df = pd.DataFrame(all_years[year]['balance_sheet'].items(), columns=['Variable', 'Value'])
    
    income_statement_ws.cell(row=1, column=current_col, value=f"Income Statement {year}")
    
    for r, row in enumerate(dataframe_to_rows(income_statement_df, index=False, header=True)):
        for c, cell_value in enumerate(row):
            income_statement_ws.cell(row=r + 2, column=current_col + c, value=cell_value)

    balance_sheet_ws.cell(row=1, column=current_col, value=f"Balance Sheet {year}")
    
    for r, row in enumerate(dataframe_to_rows(balance_sheet_df, index=False, header=True)):
        for c, cell_value in enumerate(row):
            balance_sheet_ws.cell(row=r + 2, column=current_col + c, value=cell_value)

    # Update the current column with a 1 column gap between tables
    current_col += len(income_statement_df.columns) + 1

# Save the Excel workbook
workbook.save('elkem_financial_data.xlsx')